In [1]:
from __future__ import print_function, division
import torch

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import os

from matplotlib.pyplot import imshow
from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/home/ubuntu/anaconda3/envs/controll-ST/lib/python3.6/site-packages/matplotlib/__init__.py:1003: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/controll-ST/lib/python3.6/site-packages/matplotlib/__init__.py:1003: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
img_train_path = os.path.abspath('data/train/')
img_test_path = os.path.abspath('data/test/')
csv_train_path = os.path.abspath('data/train.csv')

In [4]:
df = pd.read_csv(csv_train_path).set_index('Image')
df.head()

,Id
Image,
0000e88ab.jpg,w_f48451c
0001f9222.jpg,w_c3d896a
00029d126.jpg,w_20df2c5
00050a15a.jpg,new_whale
0005c1ef8.jpg,new_whale


In [14]:
from sklearn.model_selection import train_test_split

new_whale_df = df[df.Id == "new_whale"] # only new_whale dataset
train_df = df[~(df.Id == "new_whale")] # no new_whale dataset, used for training
unique_labels = np.unique(train_df.Id.values)

labels_dict = dict()
labels_list = []
for i in range(len(unique_labels)):
    labels_dict[unique_labels[i]] = i
    labels_list.append(unique_labels[i])
train_df.Id = train_df.Id.apply(lambda x: labels_dict[x])
split_ratio = 0.2
train_new_df, test_new_df = train_test_split(train_df, test_size=split_ratio)

train_new_df.to_csv('./data/train_new.csv', sep=',', encoding='utf-8', header=False)
test_new_df.to_csv('./data/test_new.csv', sep=',', encoding='utf-8', header=False)

In [23]:
print(df.shape)
print(len(unique_labels)) #total no. of final classes

train_data_size = int(df.shape[0]*(1-split_ratio))
valid_data_size = df.shape[0] - train_data_size
print(train_data_size, valid_data_size)

(25361, 1)
5004
20288 5073


In [7]:
train_new_df.head(5)

,Id
Image,
603fd4218.jpg,2044
ad0b56920.jpg,4474
e9afe2a62.jpg,3212
b75579156.jpg,1833
79b102480.jpg,4353


In [8]:
#Reference taken from https://github.com/pytorch/vision/issues/81 to load CSV file as a data loader in pytorch
# and https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
import torch.utils.data as data

from PIL import Image
import os
import os.path

def default_loader(path):
    return Image.open(path).convert('RGB')

def default_flist_reader(flist):
    """
    flist format: impath,label\nimpath,label\n ...(same to caffe's filelist)
    """
    imlist = []
    with open(flist, 'r') as rf:
        for line in rf.readlines():
            impath, imlabel = line.split(',')
            imlist.append( (impath, int(imlabel)) )

    return imlist

class ImageFilelist(data.Dataset):
    def __init__(self, root, flist, transform=None, target_transform=None,
            flist_reader=default_flist_reader, loader=default_loader):
        self.root   = root
        self.imlist = flist_reader(flist)
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        impath, target = self.imlist[index]
        img = self.loader(os.path.join(self.root,impath))
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.imlist)

In [9]:
import os
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform

normalize = transforms.Normalize( mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# preprocess_train = transforms.Compose([transforms.RandomResizedCrop(224),
#                                     transforms.RandomHorizontalFlip(),
#                                     transforms.ToTensor(),
#                                     normalize])


train_loader = torch.utils.data.DataLoader(
         ImageFilelist(root="./data/train/", flist="./data/train_new.csv",
             transform=transforms.Compose([transforms.RandomSizedCrop(224),
                 transforms.RandomHorizontalFlip(),
                 transforms.ToTensor(), normalize,
         ])),
         batch_size=4, shuffle=True,
         num_workers=4, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
         ImageFilelist(root="./data/train/", flist="./data/test_new.csv",
             transform=transforms.Compose([transforms.RandomSizedCrop(224),
                 transforms.ToTensor(), normalize,
         ])),
         batch_size=4, shuffle=True,
         num_workers=4, pin_memory=True)


loaders_transfer = {
        "train" : train_loader,
        "test"  : test_loader
}

print(loaders_transfer['train'].dataset)

In [10]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

model_transfer = models.resnet50(pretrained=True)

for param in model_transfer.parameters():
    param.requires_grad = False
    
num_ftrs = model_transfer.fc.in_features
print(num_ftrs)
model_transfer.fc = nn.Linear(num_ftrs, len(unique_labels))

use_cuda = torch.cuda.is_available()
# if GPU is available, move the model to GPU
if use_cuda:
    model_transfer.cuda()

2048


In [24]:
import numpy as np
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train() #put model in training mode        
        for batch_idx, (data, target) in enumerate(loaders.get('train')):
#             print(batch_idx)
#             print(batch_idx)

#             data = data.unsqueeze(0)
#             target = torch.tensor(target)
#             target = target.unsqueeze(0)
#             print(data)
#             print(target)
             # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            ## record the average training loss, using something like
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        print('validate the model')
            
        ######################    
        # validate the model #
        ######################
        model.eval() #put model in eval mode
        for batch_idx, (data, target) in enumerate(loaders.get('test')):
#             data = data.unsqueeze(0)
#             target = torch.tensor(target)
#             target = target.unsqueeze(0)
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
             # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        # calculate average losses
        train_loss = train_loss/train_data_size
        valid_loss = valid_loss/valid_data_size
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss    
    # return trained model
    return model

In [25]:
import torch.optim as optim

# learning_rate = 1e-4
criterion_transfer = nn.CrossEntropyLoss().cuda() if use_cuda else nn.CrossEntropyLoss()
optimizer_transfer = torch.optim.Adam(model_transfer.fc.parameters(), lr=0.001)

In [26]:
n_epochs = 100
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

validate the model
Epoch: 1 	Training Loss: 0.000546 	Validation Loss: 0.002686
Validation loss decreased (inf --> 0.002686).  Saving model ...
validate the model
Epoch: 2 	Training Loss: 0.000490 	Validation Loss: 0.003093
validate the model
validate the model
Epoch: 4 	Training Loss: 0.000483 	Validation Loss: 0.003218
validate the model
Epoch: 5 	Training Loss: 0.000479 	Validation Loss: 0.003224
validate the model
Epoch: 6 	Training Loss: 0.000473 	Validation Loss: 0.003276
validate the model
Epoch: 7 	Training Loss: 0.000471 	Validation Loss: 0.003561
validate the model
Epoch: 8 	Training Loss: 0.000470 	Validation Loss: 0.003612
validate the model
Epoch: 9 	Training Loss: 0.000463 	Validation Loss: 0.003558
validate the model
Epoch: 10 	Training Loss: 0.000463 	Validation Loss: 0.003580
validate the model
Epoch: 11 	Training Loss: 0.000460 	Validation Loss: 0.003797
validate the model
Epoch: 12 	Training Loss: 0.000455 	Validation Loss: 0.003745
validate the model
Epoch: 13 	Trai